In [1]:
import numpy as np
import gomoku, player_basic

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

# Playtest FeaturePlayer
Play against computer

In [2]:
game = gomoku.Gomoku(15)
opponent = player_basic.FeaturePlayer('cp', -1) # human goes first, cp goes second

In [4]:
game.play(8, 7) # human move
if not game.finished:
    game.play(*opponent.play(game)) # cp plays random move
game.show()

black played (8, 7).
white played (6, 7).
black's turn.
   0 1 2 3 4 5 6 7 8 91011121314
 0 . . . . . . . . . . . . . . . 
 1 . . . . . . . . . . . . . . . 
 2 . . . . . . . . . . . . . . . 
 3 . . . . . . . . . . . . . . . 
 4 . . . . . . . . . . . . . . . 
 5 . . . . . . . . . . . . . . . 
 6 . . . . . . . ○ . . . . . . . 
 7 . . . . . . ○ ● ● . . . . . . 
 8 . . . . . . . . . . . . . . . 
 9 . . . . . . . . . . . . . . . 
10 . . . . . . . . . . . . . . . 
11 . . . . . . . . . . . . . . . 
12 . . . . . . . . . . . . . . . 
13 . . . . . . . . . . . . . . . 
14 . . . . . . . . . . . . . . . 


Watch computers play

In [5]:
game = gomoku.Gomoku(15)
p1 = player_basic.FeaturePlayer('p1', +1)
p2 = player_basic.FeaturePlayer('p2', -1)
pa, pb = p1, p2

In [6]:
#if not game.finished:
while not game.finished:
    game.play(*pa.play(game)) # cp plays random move
    pa, pb = pb, pa
    
game.show()

black played (4, 10).
white played (9, 10).
game has ended. winner: white
   0 1 2 3 4 5 6 7 8 91011121314
 0 . . . . . . . . . . . . . . . 
 1 . . . . . . . . . . . . . . . 
 2 . . . . . . . . . . . . . . . 
 3 . . . . . . . . . . . . . . . 
 4 . . . . . . . . . . . . . . . 
 5 . . . . . . . . . . . . . . . 
 6 . . . ● . . . . . ● . . . . . 
 7 . . . . ○ ● ● ● ○ . . . . . . 
 8 . . . . . ○ ● ○ ● . . . . . . 
 9 . . . . ● ○ ○ ○ ○ ● . . . . . 
10 . . . . ● ○ ○ ○ ○ ○ . . . . . 
11 . . . . ● ○ . ● ● . . . . . . 
12 . . . . . ● . . . . . . . . . 
13 . . . . . . . . . . . . . . . 
14 . . . . . . . . . . . . . . . 


# Testing PGFeaturePlayer
Note: PGFeaturePlayer does not work and has been removed. The training code is left below for reference.

In [ ]:
# p1 = player_basic.PGFeaturePlayer('player1', 0, epsilon=0.2, discount=.5)
# p2 = player_basic.PGFeaturePlayer('player2', 0, epsilon=0.2, discount=.5)
# p2.w = p1.w
# #p1.theta = np.array([1, 4, 9, 16, 1, 4, 9, 16, 0], dtype=float)
# p2.theta = p1.theta

In [ ]:
# for i in range(1000):
#     if i%10 == 0:
#         print(i, end=', ')
#     game = gomoku.Gomoku(9)
#     p1color = np.random.choice([1, -1])
#     p1.piece = p1color
#     p2.piece = -p1color

#     players = [p1, p2]
#     turn = (1-p1color)//2
#     while not game.finished:
#         game.play(*players[turn].play(game))
#         turn = (turn+1)%2
    
#     if game.winner == p1color:
#         p1_grads = p1.get_grad(game, game._moves_left)
#         p2_grads = p2.get_grad(game, 0)
#     elif game.winner == -p1color:
#         p1_grads = p1.get_grad(game, 0)
#         p2_grads = p2.get_grad(game, game._moves_left)
#     else:
#         p1_grads = p1.get_grad(game, 0)
#         p2_grads = p2.get_grad(game, 0)
        
#     p1.step(*p1_grads)
#     p1.step(*p2_grads)
        